# This Brilliant Work could not have been possible were it not of CostiaB
[CostiaB](github.com/CostiaB)

In [1]:
#pip install -q gensim

In [2]:
import re
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.linear_model import LogisticRegression,RidgeClassifier
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.model_selection import GridSearchCV,cross_val_score,train_test_split,StratifiedKFold
from sklearn.preprocessing import FunctionTransformer
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.naive_bayes import BernoulliNB, GaussianNB, MultinomialNB
from xgboost import XGBClassifier
from sklearn.metrics import f1_score, confusion_matrix, classification_report

from imblearn.over_sampling import SMOTE

import gensim
from gensim.models.fasttext import FastText 

In [5]:
train = pd.read_csv("./src/train.csv")
display(train.shape,train.head())

(1436, 3)

,ID,Text,Label
0,ID_AASHwXxg,Mwangonde: Khansala wachinyamata Akamati achi...,POLITICS
1,ID_AGoFySzn,MCP siidakhutire ndi kalembera Chipani cha Ma...,POLITICS
2,ID_AGrrkBGP,Bungwe la MANEPO Lapempha Boma Liganizire Anth...,HEALTH
3,ID_AIJeigeG,Ndale zogawanitsa miyambo zanyanya Si zachile...,POLITICS
4,ID_APMprMbV,Nanga wapolisi ataphofomoka? Masiku ano sichi...,LAW/ORDER


In [ ]:
train.Text[6]

In [ ]:
test = pd.read_csv("../src/test.csv")
display(test.shape,test.head())

In [ ]:
train.Label.value_counts().plot.bar(figsize=(9, 3))
plt.title('Classes distribution');

In [ ]:
def preprosses_text(doc):
  # remove punctuation signs and multiple spaces
  regex_punc = re.compile('[%s]'%re.escape('.,!?'))
  doc = re.sub( regex_punc, ' ',doc)
  doc = doc.strip().lower() 
  regex_seq_spaces = re.compile(' {2,}')
  doc = re.sub( regex_seq_spaces, ' ',doc)
  return doc

train['text'] = train.Text.apply(lambda x: [i for i in preprosses_text(x).split() if len(i) > 1])
test['text'] = test.Text.apply(lambda x: [i for i in preprosses_text(x).split() if len(i) > 1])
display(train.head(3),test.head(2))

In [ ]:
# size=60, window=30, min_count=1, workers=12, iter=20, sg=1, bucket=400_000

In [ ]:
%%time
model = FastText(train.text.to_list() + test.text.to_list() ,
                 #size=60, 
                 window=30, min_count=1, workers=12,
                 #iter=20,
                 sg=1, bucket=400_000)
model.save('ft.model')

In [ ]:
def word_averaging(wv, words):
    mean = np.zeros((wv.vector_size,))
    for word in words:
        mean += wv.get_vector(word)
    mean = gensim.matutils.unitvec(mean)
    return mean

In [ ]:
%%time
union = FeatureUnion([
    ("char15", TfidfVectorizer(ngram_range =(1,5), analyzer='char_wb',stop_words=['ndi'],max_df = 0.2 ,min_df = 0.0 , max_features = 10000)),
    ("word12", TfidfVectorizer(ngram_range =(1,2), stop_words=['ndi'], max_df = 0.15,min_df = 0.01,max_features = 5000)),
    ('fast_vec', FunctionTransformer(lambda df: df.apply(lambda x: word_averaging(model.wv,[i for i in preprosses_text(x).split() if len(i) > 1])).tolist(),
                                              validate=False)), ])
pipe = Pipeline([
    ('feats', union),
    ('clf',LogisticRegression( C = 5,solver = 'liblinear',class_weight='balanced'))
    ])

res = cross_val_score(pipe, train['Text'],train['Label'],
                            cv=StratifiedKFold(5, 
                                   shuffle=True, 
                                   random_state=777))

print(res,'\n mean:{:.3f} std :{:.3f}'.format(res.mean(),res.std()))

In [ ]:
pipe.fit( train['Text'],train['Label'])

test.loc[:,"Label"] = pipe.predict(test['Text'])

test[['ID','Label']].to_csv("subww_bal.csv",index=False) 

#score 0.654838709677419 on private leaderboard